## Statistics summary by sector

In [ ]:
for i in sector:
    min_ = []
    p1 = []
    p5 = []
    p10 = []
    q1 = []
    median_ = []
    q3 = []
    p90 = []
    p95 =[]
    p99 = []
    max_ = []
    mean_ = []
    skew = []
    kurt = []
    kurt += list(complete[complete['Sector'] == i].kurtosis(axis = 0, skipna = True))    
    skew += list(complete[complete['Sector'] == i].skew(axis = 0, skipna = True)) 
    p1 += list(complete[complete['Sector'] == i].quantile(0.01, numeric_only=True)) 

    p5 += list(complete[complete['Sector'] == i].quantile(0.05, numeric_only=True)) 

    p10 += list(complete[complete['Sector'] == i].quantile(0.1, numeric_only=True))
 
    p90 += list(complete[complete['Sector'] == i].quantile(0.90, numeric_only=True)) 

    p95 += list(complete[complete['Sector'] == i].quantile(0.95, numeric_only=True))

    p99 += list(complete[complete['Sector'] == i].quantile(0.99, numeric_only=True))

    min_ += list(complete[complete['Sector'] == i].min(axis = 0, numeric_only=True))

    max_ += list(complete[complete['Sector'] == i].max(axis = 0, numeric_only=True))

    mean_ += list(complete[complete['Sector'] == i].mean(axis = 0, numeric_only=True))

    median_ += list(complete[complete['Sector'] == i].median(axis = 0, numeric_only=True))

    q1 += list(complete[complete['Sector'] == i].quantile(0.25)) 

    q3 += list(complete[complete['Sector'] == i].quantile(0.75)) 

    
    data0={'variable':variables,'min':min_,'P1':p1,'P5':p5, 'P10':p10, 'Q1':q1, 'median':median_, 'Q3':q3,'P90':p90,
          'P95':p95, 'P99':p99,'max':max_, 'mean':mean_,'skew':skew,'kurt':kurt}
    data1 = pd.DataFrame(data0)
    df1 = complete[complete['Sector'] == i]
    zero_ = list()
    positive_ = list()
    negative_ = list()
    miss_ = list()
    cov = list()
    for n in variables:
        zero_.append(len(df1[df1[n] == 0]))
        positive_.append(len(df1[df1[n] > 0]))
        negative_.append(len(df1[df1[n] < 0]))
        miss_.append(len(df1[df1[n].isnull()]))
        cov.append(len(df1[df1[n].notnull()]) / len(df1))
    data1['zero'] = zero_
    data1['positive'] = positive_
    data1['negative'] = negative_
    data1['missing'] = miss_
    data1['coverage'] = cov
    
    data1.to_csv(f"summary_{i}_6.csv")

## Overall statistics summary

In [ ]:
min_ = []
p1 = []
p5 = []
p10 = []
q1 = []
median_ = []
q3 = []
p90 = []
p95 =[]
p99 = []
max_ = []
mean_ = []
skew = []
kurt = []
zero_ = list()
positive_ = list()
negative_ = list()
miss_ = list()
cov = list()
kurt += list(complete.kurtosis(axis = 0, skipna = True))

skew += list(complete.skew(axis = 0, skipna = True)) 

p1 += list(complete.quantile(0.01, numeric_only=True)) 

p5 += list(complete.quantile(0.05, numeric_only=True)) 

p10 += list(complete.quantile(0.1, numeric_only=True))

p90 += list(complete.quantile(0.90, numeric_only=True)) 

p95 += list(complete.quantile(0.95, numeric_only=True))

p99 += list(complete.quantile(0.99, numeric_only=True))

min_ += list(complete.min(axis = 0, numeric_only=True))

max_ += list(complete.max(axis = 0, numeric_only=True))

mean_ += list(complete.mean(axis = 0, numeric_only=True))

median_ += list(complete.median(axis = 0, numeric_only=True))

q1 += list(complete.quantile(0.25)) 

q3 += list(complete.quantile(0.75)) 

for n in variables:
        zero_.append(len(complete[complete[n] == 0]))
        positive_.append(len(complete[complete[n] > 0]))
        negative_.append(len(complete[complete[n] < 0]))
        miss_.append(len(complete[complete[n].isnull()]))
        cov.append(len(complete[complete[n].notnull()]) / len(complete))
data00={'variable':variables,'coverage':cov,'min':min_,'P1':p1,'P5':p5, 'P10':p10, 'Q1':q1, 'median':median_, 'Q3':q3,'P90':p90,
      'P95':p95, 'P99':p99,'max':max_, 'mean':mean_,'skew':skew,'kurt':kurt,'zero':zero_,'positive':positive_,
      'negative':negative_, "missing":miss_}
data2 = pd.DataFrame(data00)
data2.to_csv('overall_summary_5.csv')

## Event rate by sector

In [ ]:
# four category by sector
complete= pd.read_excel('D&B dataset.xlsx')
columns1 = ['subscriberCount','videoCount','liked','disliked','views','comment','video_2017','video_2018','first']
sector = ['Basic Materials', 'Communication Services', 'Consumer Cyclical',
       'Consumer Defensive', 'Energy', 'Financial Services','Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']
for i in sector:
    youtube1 = complete[complete['Sector'] == i][['subscriberCount','videoCount','liked','disliked','views','comment','video_2017','video_2018','first']]
    df1 = pd.read_csv(f"summary_{i}_6.csv", index_col="variable").drop('Unnamed: 0', axis = 1)
    for a in columns1:
        for b in youtube1.index:
            if pd.isnull(youtube1[a][b]):
                complete[a][b] = 0
            elif youtube1[a][b] > df1['P90'][a]:
                complete[a][b] = 3
            elif youtube1[a][b] < df1['Q1'][a]:
                complete[a][b] = 1
            else:
                 complete[a][b] = 2
# export files                    
for a in columns1:
    grouped = pd.DataFrame(complete.groupby([a, 'Sector','margin_change_90']).size().unstack())
    grouped = grouped.replace(np.nan, 0)
    grouped['sum'] = grouped[0.0] + grouped[1.0]
    grouped['event_rate'] = grouped[1.0]/grouped['sum']
    grouped.to_excel(f"sector_{a}_event rate.xlsx")

## WOE by sector

In [ ]:
# read data
import pandas as pd
import numpy as np 
from pandas import read_excel
complate= pd.read_excel('D&B dataset.xlsx')
complete = complete.set_index('ticker')
event_rate_sector =read_excel("update sector event rate_0410.xlsx")
event_rate = event_rate_sector[['Cate','Sector','subscriberCount','videoCount','liked','disliked','views','comment','video_2017','video_2018','first']]

In [ ]:
import math
columns1 = ['subscriberCount','videoCount','liked','disliked','views','comment','video_2017','video_2018','first']
sector = ['Basic Materials', 'Communication Services', 'Consumer Cyclical',
       'Consumer Defensive', 'Energy', 'Financial Services','Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']
for i in sector:
    youtube1 = complete[complete['Sector'] == i][['subscriberCount','videoCount','liked','disliked','views','comment','video_2017','video_2018','first']]
    df1 = pd.read_csv(f"summary_{i}_6.csv", index_col="variable").drop('Unnamed: 0', axis = 1)
    event_rate2 = event_rate[event_rate['Sector'] ==i]
    for a in columns1:
        for b in youtube1.index:
            complete[a][b] = np.nan
            if pd.isnull(youtube1[a][b]):
                eventrate = event_rate2[event_rate2['Cate'] ==0][a].values[0]
                woe = math.log((1-eventrate) / eventrate)
                if math.isinf(woe) or math.isnan(woe):
                    complete[a][b] = 0
                else:
                    complete[a][b] = woe
            elif youtube1[a][b] > df1['P90'][a]:
                eventrate = event_rate2[event_rate2['Cate'] ==3][a].values[0]
                woe = math.log((1-eventrate) / eventrate)
                if math.isinf(woe) or math.isnan(woe):
                    complete[a][b] = 0
                else:
                    complete[a][b] = woe
            
            elif youtube1[a][b] < df1['Q1'][a]:
                eventrate =  event_rate2[event_rate2['Cate'] ==1][a].values[0]
                woe = math.log((1-eventrate) / eventrate)
                if math.isinf(woe) or math.isnan(woe):
                    complete[a][b] = 0
                else:
                    complete[a][b] = woe
            else:
                eventrate = event_rate2[event_rate2['Cate'] ==2][a].values[0]
                woe = math.log((1-eventrate) / eventrate)
                if math.isinf(woe) or math.isnan(woe):
                    complete[a][b] = 0
                else:
                    complete[a][b] = woe

complete.to_excel('woe by sector.xlsx')

# Company profile analysis

In [ ]:
# High group and low group
import pandas as pd
from pandas import read_excel
data = pd.read_excel('D&B dataset.xlsx')
df = pd.read_excel('tree_90_0413.xlsx',sheet_name = 'dataset_leaf')
leaf_data = df[['ticker','leaf']]
data_1 = pd.merge(data,leaf_data,how = 'inner',on = 'ticker') 
group_high_plain = data_1[data_1['leaf'] == 13]
group_low_plain = data_1[data_1['leaf'] == 8]

## word cloud

In [ ]:
! pip install wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
low_list = ''
for i in group_low_plain['Key_Words'].tolist():
    low_list += i

high_list = ''
for i in group_high_plain['Key_Words'].tolist():
    high_list += i

In [ ]:
# low group
stopwords_low = set(STOPWORDS)
stopwords_low.update(["including"])
wordcloud = WordCloud(stopwords=stopwords_low,max_font_size=50, max_words=80, background_color="white").generate(low_list)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# high group
stopwords = set(STOPWORDS)
stopwords.update(["including", "fixed", "approximately","headquartered"])
wordcloud = WordCloud(stopwords=stopwords,max_font_size=50, max_words=80, background_color="white").generate(high_list)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## industry distribution

In [ ]:
plt.hist([group_high_plain['Sector'], group_low_plain['Sector']], bins=11, label=['High Group', 'Low Group'])
plt.legend(loc='upper right')
plt.xticks(rotation=90)
plt.ylabel('Frequency',fontsize=13)
plt.xlabel('Industry',fontsize=13)
plt.title('Industry Distribution',fontsize=15)
plt.show()

## location distribution

In [ ]:
plt.figure(figsize=(20,10))
plt.hist([group_high_plain[group_high_plain['country'] == 'United States']['states'], group_low_plain[group_low_plain['country'] == 'United States']['states']], bins=53, label=['High Group', 'Low Group'])
plt.legend(loc='upper right')
plt.xticks(rotation=0, fontsize=12)
plt.ylabel('Frequency',fontsize=13)
plt.xlabel('Company Location',fontsize=13)
plt.title('Company Location Distribution',fontsize=15)

## company size distribution

In [ ]:
plt.hist([group_high_plain['number_of_full-time_employee'], group_low_plain['number_of_full-time_employee']], bins='auto', label=['High Group', 'Low Group'])
plt.legend(loc='upper right')
plt.xticks(rotation=90)
plt.ylabel('Frequency',fontsize=13)
plt.xlabel('Company Size',fontsize=13)
plt.title('Company Size Distribution',fontsize=15)
plt.xlim(1, 20000)
plt.show()

## hypothesis test

In [ ]:
# Ho: group high = group low
# H1: group high != group low
from scipy.stats import ttest_ind
ttest,pval = ttest_ind(high_diff.dropna()['number_of_full-time_employee'],low_diff.dropna()['number_of_full-time_employee'])
pval